<a href="https://colab.research.google.com/github/anuradha1105/unsloth/blob/main/Unsloth_ai_colab_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# environment setup

In [ ]:
!nvidia-smi
import torch, sys
print("Python:", sys.version)
print("Torch:", torch.__version__ if hasattr(torch, "__version__") else "Not installed")


Sun Nov  9 13:14:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Fresh start
%pip -q install --upgrade pip

# Install PyTorch CUDA 12.4 wheels
%pip -q install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio
import torch, platform
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available(), "| Device cap:", torch.cuda.get_device_capability() if torch.cuda.is_available() else None)


Torch: 2.5.1+cu124 | CUDA available: True | Device cap: (7, 5)


In [ ]:
%pip -q install unsloth


In [ ]:
# For Torch 2.5 + CUDA 12.4 wheels installed above:
%pip -q install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!wget -qO- https://raw.githubusercontent.com/unslothai/unsloth/main/unsloth/_auto_install.py | python -


pip install --upgrade pip && pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"


In [ ]:
%pip install --upgrade --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
%pip install --upgrade --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth-zoo.git


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-pwjii9df
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-pwjii9df
  Resolved https://github.com/unslothai/unsloth.git to commit 1c0ad844f170f67c7cdf6f7a9465bafb0f9627df
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.11.2-py3-none-any.whl size=355531 sha256=b08544de79774b6aede66c3d28064b4ee0cbb0e366410e01d9c672f82ce729a0
  Stored in directory: /tmp/pip-ephem-wheel-cache-vulo7ctb/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.11.2
    Uninstalling unsloth-2025.11.2:
      Successfully uninstalled unsloth-2025.11.2
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-req-build-s

In [ ]:
from unsloth import FastLanguageModel, FastModel
import torch
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

max_seq_length = 2048

# Tiny public jsonl for a quick smoke test
url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
dataset = load_dataset("json", data_files={"train": url}, split="train")

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",   # pick any supported model
    max_seq_length = max_seq_length,
    load_in_4bit = True,                  # QLoRA 4-bit
    load_in_8bit = False,
    load_in_16bit = False,
    full_finetuning = False,
    # token="hf_..."                       # if using gated models
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,
    loftq_config = None,
)

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    args = SFTConfig(
        max_seq_length = max_seq_length,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,           # use num_train_epochs=1..3 for a real run
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)

trainer.train()


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


ImportError: cannot import name 'is_mistral_common_available' from 'transformers.utils' (/usr/local/lib/python3.12/dist-packages/transformers/utils/__init__.py)

In [ ]:
!nvidia-smi
import sys; print("Python:", sys.version)


Sun Nov  9 13:20:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%pip -q install --upgrade pip
%pip -q install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio
import torch; print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| GPU available:", torch.cuda.is_available())


Torch: 2.5.1+cu124 | CUDA: 12.4 | GPU available: True


In [ ]:
# 0) Info (optional)
!nvidia-smi
import sys; print("Python:", sys.version)


Sun Nov  9 13:23:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 1) Torch (CUDA 12.4 wheels are stable on Colab)
%pip -q install --upgrade pip
%pip -q install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio
import torch; print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| GPU:", torch.cuda.is_available())


Torch: 2.5.1+cu124 | CUDA: 12.4 | GPU: True


In [ ]:
# Create a clean virtual environment
!python -m venv /content/unsloth-venv
# Upgrade pip inside the venv
!/content/unsloth-venv/bin/python -m pip install --upgrade pip


Error: Command '['/content/unsloth-venv/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/content/unsloth-venv/bin/python: No module named pip


In [ ]:
!rm -rf /content/unsloth-venv


In [ ]:
!python -m venv /content/unsloth-venv --without-pip


In [ ]:
!curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!/content/unsloth-venv/bin/python get-pip.py


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)


In [ ]:
# Torch (CUDA 12.4)
!/content/unsloth-venv/bin/python -m pip install --index-url https://download.pytorch.org/whl/cu124 torch==2.5.1+cu124

# Unsloth + dependencies
!/content/unsloth-venv/bin/python -m pip install \
  "trl==0.23.0" "transformers==4.56.2" \
  "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.45.0" "datasets>=2.20.0" \
  unsloth unsloth_zoo


Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached https://download.pytorch.org/whl/cu124/torch-2.5.1%2Bcu124-cp312-cp312-linux_x86_64.whl (908.2 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (24.6 MB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (883 kB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_cup

In [ ]:
import os, sys, glob, site

# Path to your venv
venv_path = "/content/unsloth-venv"

# Compute the venv's site-packages folder
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"

# If folder name differs (e.g., python3.10 vs 3.12), auto-detect it
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands:
    site_dir = cands[0]

# Add it so imports come from the venv
site.addsitedir(site_dir)
print("✅ Added venv to sys.path:", site_dir)

# ---- Sanity check ----
import torch, transformers, trl, unsloth
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("TRL:", trl.__version__)
print("Unsloth imported OK from:", unsloth.__file__)


✅ Added venv to sys.path: /content/unsloth-venv/lib/python3.12/site-packages


/tmp/ipython-input-400182861.py:20: UserWarning: WARNING: Unsloth should be imported before trl, transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import torch, transformers, trl, unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Torch: 2.8.0+cu128
Transformers: 4.57.1
TRL: 0.23.0
Unsloth imported OK from: /usr/local/lib/python3.12/dist-packages/unsloth/__init__.py


In [ ]:
!nvidia-smi
import torch
print("GPU available:", torch.cuda.is_available())


Sun Nov  9 13:45:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             28W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from unsloth import FastLanguageModel, FastModel
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

max_seq_length = 1024

# Small public dataset for a quick test
dataset = load_dataset(
    "json",
    data_files={"train": "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"},
    split="train[:200]"
)

# Load base model (you can later change to Llama, Gemma, etc.)
model, tokenizer = FastModel.from_pretrained(
    model_name      = "unsloth/gpt-oss-20b",
    max_seq_length  = max_seq_length,
    load_in_4bit    = True,      # fits on T4
    full_finetuning = False,
)

# Apply LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=max_seq_length,
)

# Configure trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=SFTConfig(
        output_dir="outputs",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        max_steps=40,      # quick run
        logging_steps=2,
        optim="adamw_8bit",
        seed=3407,
        max_seq_length=max_seq_length,
    ),
)

trainer.train()


unified_chip2.jsonl:   0%|          | 0.00/95.6M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

==((====))==  Unsloth 2025.11.2: Fast Gpt_Oss patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anuradhasrivastav25 (anuradhasrivastav25-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
2,3.246800
4,2.982800
6,2.773100
8,2.457800
10,1.911100
12,1.672400
14,1.973900
16,2.150000
18,1.673500
20,1.773400


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=40, training_loss=2.0534964919090273, metrics={'train_runtime': 1153.5599, 'train_samples_per_second': 0.277, 'train_steps_per_second': 0.035, 'total_flos': 6124558737604608.0, 'train_loss': 2.0534964919090273, 'epoch': 1.6})

In [ ]:
from datasets import load_dataset
ds = load_dataset("tatsu-lab/alpaca", split="train[:5000]")  # take small slice for speed
ds = ds.train_test_split(test_size=0.1, seed=42)
train_ds, eval_ds = ds["train"], ds["test"]

def format_alpaca(example):
    instruction = example["instruction"].strip()
    input_text = example["input"].strip()
    output = example["output"].strip()
    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
    return {"text": prompt}

train_data = train_ds.map(format_alpaca, remove_columns=train_ds.column_names)
eval_data  = eval_ds.map(format_alpaca,  remove_columns=eval_ds.column_names)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
!/content/unsloth-venv/bin/python -m pip install unsloth unsloth_zoo


/bin/bash: line 1: /content/unsloth-venv/bin/python: No such file or directory


In [ ]:
# === Recreate Unsloth venv cleanly ===
!python -m venv /content/unsloth-venv --without-pip
!curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!/content/unsloth-venv/bin/python get-pip.py
!/content/unsloth-venv/bin/python -m pip install --upgrade pip setuptools wheel

# Install PyTorch (CUDA 12.4 wheel)
!/content/unsloth-venv/bin/python -m pip install --index-url https://download.pytorch.org/whl/cu124 torch==2.5.1+cu124

# Install Unsloth + compatible stack
!/content/unsloth-venv/bin/python -m pip install \
  "trl==0.23.0" "transformers==4.56.2" \
  "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.45.0" "datasets>=2.20.0" \
  unsloth unsloth_zoo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [setuptools]
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 14.1 MB/s  0:00:26
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 151.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 136.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 15.9 MB/s  0:00:19
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 18.5 MB/s  0:00:14
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 34.5 MB/s  0:00:06
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 68.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 66.6 MB/s  0:00:01
     ━━━━━━━━━━━

In [ ]:
import os, sys, glob, site

venv_path = "/content/unsloth-venv"
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands:
    site_dir = cands[0]
site.addsitedir(site_dir)
print("✅ Added venv to sys.path:", site_dir)

# Sanity check
import torch, transformers, trl, unsloth
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("TRL:", trl.__version__)
print("Unsloth imported OK from:", unsloth.__file__)


✅ Added venv to sys.path: /content/unsloth-venv/lib/python3.12/site-packages


/tmp/ipython-input-2057073134.py:13: UserWarning: WARNING: Unsloth should be imported before trl, transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import torch, transformers, trl, unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Torch: 2.8.0+cu126
Transformers: 4.57.1
TRL: 0.23.0
Unsloth imported OK from: /content/unsloth-venv/lib/python3.12/site-packages/unsloth/__init__.py


In [ ]:
from unsloth import FastLanguageModel, FastModel
from trl import SFTTrainer, SFTConfig
# ... rest of the fine-tuning code


# Full Finetuning (smollm2-135m) on Alpaca subset

In [ ]:
!nvidia-smi
import torch; print("CUDA:", torch.cuda.is_available())


Sun Nov  9 14:33:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P0             26W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from datasets import load_dataset

# Small, fast slice for first run. Later increase to "train[:100%]".
raw = load_dataset("tatsu-lab/alpaca", split="train[:5000]", cache_dir="/content/.cache_hf")
splits = raw.train_test_split(test_size=0.1, seed=42)
train_ds, eval_ds = splits["train"], splits["test"]

def format_alpaca(ex):
    instr = (ex.get("instruction") or "").strip()
    inp   = (ex.get("input") or "").strip()
    out   = (ex.get("output") or "").strip()
    user  = f"### Instruction:\n{instr}" + (f"\n\n### Input:\n{inp}" if inp else "")
    return {"text": f"{user}\n\n### Response:\n{out}"}

train_data = train_ds.map(format_alpaca, remove_columns=train_ds.column_names)
eval_data  = eval_ds.map(format_alpaca,  remove_columns=eval_ds.column_names)

print(train_data[0]["text"][:500])
print(f"Train/Eval sizes: {len(train_data)}/{len(eval_data)}")


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

### Instruction:
Classify this sentence as True or False.

### Input:
The capital of Spain is Madrid.

### Response:
True
Train/Eval sizes: 4500/500


In [ ]:
from unsloth import FastModel
from trl import SFTTrainer, SFTConfig

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = False,     # FULL fine-tune (no quantization)
    full_finetuning = True,      # <— key difference vs LoRA
)

args = SFTConfig(
    output_dir="outputs_full",
    per_device_train_batch_size=8,     # small model → can go higher; drop if OOM
    gradient_accumulation_steps=1,
    num_train_epochs=1,                # or set max_steps=1000 for time-boxed training
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    optim="adamw_torch",               # standard AdamW
    learning_rate=2e-4,                # small model tolerates a bit higher LR
    weight_decay=0.01,
    seed=42,
    max_seq_length=MAX_LEN,
    dataset_text_field="text",
    fp16=True,                         # T4 = FP16 (if bf16=False)
    bf16=False,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=eval_data,
    args=args,
)

trainer.train()


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Float16 full finetuning uses more memory since we upcast weights to float32.


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anuradhasrivastav25 (anuradhasrivastav25-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss,Validation Loss
200,1.720800,1.664466
400,1.635800,1.613969


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=563, training_loss=1.6760839408074983, metrics={'train_runtime': 470.1511, 'train_samples_per_second': 9.571, 'train_steps_per_second': 1.197, 'total_flos': 503722991808000.0, 'train_loss': 1.6760839408074983, 'epoch': 1.0})

In [ ]:
model.save_pretrained("finetuned_full_smollm2")
tokenizer.save_pretrained("finetuned_full_smollm2")
!ls -lah finetuned_full_smollm2


total 518M
drwxr-xr-x 2 root root 4.0K Nov  9 14:44 .
drwxr-xr-x 1 root root 4.0K Nov  9 14:43 ..
-rw-r--r-- 1 root root   29 Nov  9 14:44 added_tokens.json
-rw-r--r-- 1 root root  795 Nov  9 14:43 config.json
-rw-r--r-- 1 root root  168 Nov  9 14:43 generation_config.json
-rw-r--r-- 1 root root 456K Nov  9 14:44 merges.txt
-rw-r--r-- 1 root root 514M Nov  9 14:44 model.safetensors
-rw-r--r-- 1 root root  856 Nov  9 14:44 special_tokens_map.json
-rw-r--r-- 1 root root 4.0K Nov  9 14:44 tokenizer_config.json
-rw-r--r-- 1 root root 3.4M Nov  9 14:44 tokenizer.json
-rw-r--r-- 1 root root 782K Nov  9 14:44 vocab.json


In [ ]:
# Save the fully fine-tuned model and tokenizer
SAVE_DIR = "full_smollm2_final"
trainer.save_model(SAVE_DIR)           # saves model weights + config
tokenizer.save_pretrained(SAVE_DIR)    # saves tokenizer vocab + config
print("Saved to:", SAVE_DIR)


Saved to: full_smollm2_final


# (LoRA) on the same dataset (smaller memory, faster)

In [ ]:
from unsloth import FastLanguageModel, FastModel
from trl import SFTTrainer, SFTConfig

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = True,      # LoRA path (QLoRA)
    full_finetuning = False,     # <— enables PEFT
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)

args = SFTConfig(
    output_dir="outputs_lora",
    per_device_train_batch_size=8,        # should be fine; lower if OOM
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    optim="adamw_8bit",                   # 8-bit optimizer
    learning_rate=2e-4,
    weight_decay=0.01,
    seed=42,
    max_seq_length=MAX_LEN,
    dataset_text_field="text",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=eval_data,
    args=args,
)
trainer.train()


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model` require gradients


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,500 | Num Epochs = 1 | Total steps = 563
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


Step,Training Loss,Validation Loss
200,1.730700,1.688717
400,1.680300,1.655471


TrainOutput(global_step=563, training_loss=1.768907067619039, metrics={'train_runtime': 411.9862, 'train_samples_per_second': 10.923, 'train_steps_per_second': 1.367, 'total_flos': 526890080448000.0, 'train_loss': 1.768907067619039, 'epoch': 1.0})

In [ ]:
model.save_pretrained("finetuned_lora_smollm2")
tokenizer.save_pretrained("finetuned_lora_smollm2")

prompt = "Write a concise function in Python that reverses a string."
x = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.inference_mode():
    y = model.generate(**x, max_new_tokens=120, do_sample=True, temperature=0.7)
print(tokenizer.decode(y[0], skip_special_tokens=True))


Write a concise function in Python that reverses a string.

### Input:
Song: "Mr. Solem

### Output:
Mr. Solem

### Explanation:
Song's music is solem, and Mr. Solem is solem, and so is Mr. Solem - no, they're solem, and so am I.

### Input:
Song: "Mr. Solem"

### Output:
Mr. Solem

### Explanation:
Song's music is solem, and so is Mr. Solem - no, they're solem


In [ ]:
SAVE_DIR = "lora_smollm2_final"
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("Saved to:", SAVE_DIR)


Saved to: lora_smollm2_final


GRPO (reasoning) with TRL’s GRPOTrainer

In [ ]:
!/content/unsloth-venv/bin/python -m pip -q install "trl==0.24.0"
import os, sys, glob, site
venv_path = "/content/unsloth-venv"
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands: site_dir = cands[0]
site.addsitedir(site_dir)

import torch, transformers, trl, unsloth
from trl import GRPOTrainer, GRPOConfig
print("TRL:", trl.__version__)


TRL: 0.23.0


In [ ]:
from datasets import load_dataset

gsm = load_dataset("openai/gsm8k", "main", split="train[:1000]")  # small subset
# Fields: "question", "answer" (answer often ends with '#### final_number')
print(gsm[0])


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}


In [ ]:
import re

def extract_final_number(s: str):
    # common GSM8K pattern: "... #### 42"
    m = re.search(r"####\s*(-?\d+(?:\.\d+)?)", s)
    return m.group(1).strip() if m else None

def reward_function(samples, **kwargs):
    # samples: list[str] model generations
    # kwargs can include batch with "answer"
    rewards = []
    golds = kwargs.get("references")  # we’ll pass gold answers separately
    for gen, gold in zip(samples, golds):
        gold_num = extract_final_number(gold or "")
        got_num  = extract_final_number(gen or "")
        rewards.append(1.0 if (gold_num is not None and gold_num == got_num) else 0.0)
    return rewards


In [ ]:
from unsloth import FastLanguageModel, FastModel

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 768

policy, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = True,
    full_finetuning = False,
)
policy = FastLanguageModel.get_peft_model(
    policy,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16, lora_dropout=0.05, bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
def to_prompt(ex):
    # short, explicit instruction
    return f"""Solve the math word problem step by step, then answer with '#### <final_number>' on the last line.

Problem:
{ex['question']}
"""

prompts = [to_prompt(ex) for ex in gsm]
references = [ex["answer"] for ex in gsm]  # used by reward function


In [ ]:
# 1) Build a Dataset with prompts (and keep references separately for the reward)
from datasets import Dataset

train_ds = Dataset.from_dict({"prompt": prompts})
len(train_ds), train_ds[0]


(1000,
 {'prompt': "Solve the math word problem step by step, then answer with '#### <final_number>' on the last line.\n\nProblem:\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\n"})

In [ ]:
# assumes you already created `prompts` and `references` lists earlier
prompt_to_answer = {p: a for p, a in zip(prompts, references)}


In [ ]:
import re

def extract_final_number(s: str):
    m = re.search(r"####\s*(-?\d+(?:\.\d+)?)", s)
    return m.group(1).strip() if m else None

def reward_function(*, prompts=None, completions=None, completion_ids=None, **kwargs):
    """
    Unsloth GRPO will call this with keyword args. We use:
      - prompts:     list[str]   (batch prompts)
      - completions: list[str]   (model generations for each prompt)
    Return: list[float] rewards length == len(completions)
    """
    rewards = []
    for p, gen in zip(prompts or [], completions or []):
        gold = prompt_to_answer.get(p, "")
        got = extract_final_number(gen or "")
        ans = extract_final_number(gold or "")
        rewards.append(1.0 if (ans is not None and got == ans) else 0.0)
    return rewards


In [ ]:
import os, sys, glob, site

venv_path = "/content/unsloth-venv"
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"

# Find the venv site-packages folder automatically
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands:
    site_dir = cands[0]

site.addsitedir(site_dir)

import torch, transformers
print("✅ Attached venv at:", site_dir)
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)


✅ Attached venv at: /content/unsloth-venv/lib/python3.12/site-packages
Torch: 2.8.0+cu126
Transformers: 4.57.1


In [ ]:
# Clean venv
!rm -rf /content/unsloth-venv
!python -m venv /content/unsloth-venv --without-pip
!curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!/content/unsloth-venv/bin/python get-pip.py
!/content/unsloth-venv/bin/python -m pip install --upgrade pip setuptools wheel

# Torch (CUDA 12.4 wheel), Transformers, TRL, and Unsloth + deps
!/content/unsloth-venv/bin/python -m pip install --index-url https://download.pytorch.org/whl/cu124 torch==2.5.1+cu124
!/content/unsloth-venv/bin/python -m pip install \
  "transformers==4.56.2" "trl==0.24.0" \
  "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.45.0" "datasets>=2.20.0" \
  unsloth unsloth_zoo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [setuptools]
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 24.1 MB/s  0:00:20
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 129.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 30.6 MB/s  0:00:13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 60.5 MB/s  0:00:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 72.3 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 67.3 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 67.9 MB/s  0:00:01
     ━━━━━━━━━━━━

In [ ]:
# Remove conflicting packages from the *venv*
!/content/unsloth-venv/bin/python -m pip uninstall -y xformers torchvision

# (Optional) make sure torch & core libs are exactly what we want
!/content/unsloth-venv/bin/python -m pip install -q --index-url https://download.pytorch.org/whl/cu124 torch==2.5.1+cu124
!/content/unsloth-venv/bin/python -m pip install -q "transformers==4.56.2" "trl==0.24.0" "accelerate>=0.34.2" "peft>=0.13.2" "datasets>=2.20.0" unsloth unsloth_zoo


Found existing installation: xformers 0.0.32.post2
Uninstalling xformers-0.0.32.post2:
  Successfully uninstalled xformers-0.0.32.post2
Found existing installation: torchvision 0.23.0
Uninstalling torchvision-0.23.0:
  Successfully uninstalled torchvision-0.23.0


In [ ]:
from datasets import Dataset

# A small set of arithmetic prompts
prompts = [
    "Q: 17 + 28 = ?\nA:",
    "Q: 65 - 19 = ?\nA:",
    "Q: 7 * 8 = ?\nA:",
    "Q: 144 / 12 = ?\nA:",
    "Q: 29 + 34 = ?\nA:",
    "Q: 81 - 27 = ?\nA:",
    "Q: 9 * 9 = ?\nA:",
    "Q: 56 / 7 = ?\nA:",
    "Q: 12 + 45 = ?\nA:",
    "Q: 100 - 58 = ?\nA:",
]

# References (ground-truth answers as strings)
references = [
    "45", "46", "56", "12", "63", "54", "81", "8", "57", "42",
]

# Trainer expects a dataset with a "prompt" field
train_ds = Dataset.from_dict({"prompt": prompts})
len(train_ds), train_ds[0]


(10, {'prompt': 'Q: 17 + 28 = ?\nA:'})

In [ ]:
import re
import torch

def extract_number(text):
    # keep last number-like token
    nums = re.findall(r"-?\d+(?:\.\d+)?", text)
    return nums[-1] if nums else None

@torch.no_grad()
def reward_math(prompts, completions, completion_ids=None, **kwargs):
    refs = kwargs.get("references", [])
    rewards = []
    for i, (p, c) in enumerate(zip(prompts, completions)):
        ref = refs[i % len(refs)]  # cycle if needed
        pred = extract_number(c) or ""
        # exact match reward; partial credit if digits overlap
        if pred.strip() == ref.strip():
            r = 1.0
        elif re.sub(r"\D", "", pred) == re.sub(r"\D", "", ref) and pred != "":
            r = 0.5
        else:
            r = 0.0
        rewards.append(r)
    return torch.tensor(rewards, device="cuda", dtype=torch.float32)


In [ ]:
print("🚀 Starting GRPO reasoning training…")
trainer.train()


🚀 Starting GRPO reasoning training…


Step,Training Loss
10,0.462400
20,0.428000
30,0.537000
40,0.502900
50,0.530400
60,0.536500
70,0.460500
80,0.498500
90,0.579400
100,0.469700


TrainOutput(global_step=500, training_loss=0.5616034145355224, metrics={'train_runtime': 413.4316, 'train_samples_per_second': 4.838, 'train_steps_per_second': 1.209, 'total_flos': 0.0, 'train_loss': 0.5616034145355224, 'epoch': 1.0})

In [ ]:
def generate(model, tok, prompt):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    out = model.generate(
        **inputs,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        max_new_tokens=64,
        pad_token_id=tok.eos_token_id,
    )
    return tok.decode(out[0], skip_special_tokens=True)

test_q = "Q: 23 + 19 = ?\nA:"
print("=== After GRPO ===")
print(generate(trainer.model.eval(), tokenizer, test_q))


In [ ]:
# should print a GPU name and True
import torch, platform
print("CUDA available:", torch.cuda.is_available())
print("Torch:", torch.__version__, "| CUDA build:", torch.version.cuda)


CUDA available: True
Torch: 2.8.0+cu126 | CUDA build: 12.6


# (GRPO Reasoning)

In [ ]:
!nvidia-smi
import torch
print("✅ CUDA available:", torch.cuda.is_available())


Sun Nov  9 21:45:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -q "transformers==4.57.1" "trl==0.24.0" \
               "accelerate>=0.34.2" "peft>=0.13.2" \
               "datasets>=2.20.0" "bitsandbytes>=0.45.0" \
               unsloth unsloth_zoo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# === 1. Dataset ===
from datasets import load_dataset, Dataset
import re

gsm = load_dataset("openai/gsm8k", "main", split="train[:1000]")

def to_prompt(rec):
    return f"""Solve the math word problem step by step, then answer with '#### <final_number>' on the last line.

Problem:
{rec['question']}
"""
prompts = [to_prompt(x) for x in gsm]
answers = [x["answer"] for x in gsm]
train_ds = Dataset.from_dict({"prompt": prompts})
answer_map = dict(zip(prompts, answers))

# === 2. Reward function ===
def extract_final_number(text):
    m = re.search(r"####\s*(-?\d+(?:\.\d+)?)", text)
    return m.group(1).strip() if m else None

def reward_fn(*, prompts=None, completions=None, completion_ids=None, **kwargs):
    rewards = []
    for p, gen in zip(prompts or [], completions or []):
        gold = answer_map.get(p, "")
        got  = extract_final_number(gen or "")
        ref  = extract_final_number(gold or "")
        rewards.append(1.0 if got == ref and ref is not None else 0.0)
    return rewards

# === 3. Model (LoRA + 4-bit) ===
from unsloth import FastLanguageModel, FastModel

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN    = 768

policy, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = True,
    full_finetuning = False,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

policy = FastLanguageModel.get_peft_model(
    policy,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj",
                    "gate_proj","up_proj","down_proj"],
    lora_alpha=16, lora_dropout=0.05, bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)

# === 4. GRPO training ===
from trl import GRPOTrainer, GRPOConfig

cfg = GRPOConfig(
    output_dir="outputs_grpo",
    per_device_train_batch_size=4,   # raise to 8 on L4/A100
    gradient_accumulation_steps=1,
    max_steps=100,                   # longer = better; start small
    logging_steps=5,
    save_steps=50,
    learning_rate=5e-6,
    max_prompt_length=384,
    max_completion_length=192,
    beta=0.03,
)

trainer = GRPOTrainer(
    model=policy,
    tokenizer=tokenizer,
    args=cfg,
    reward_funcs=[reward_fn],
    train_dataset=train_ds,
    train_kwargs={},
)

print("✅ Starting GRPO training… (first step compiles kernels, be patient)")
trainer.train()


In [ ]:
# 1) Build policy (LoRA) and frozen ref from SAME BASE
from unsloth import FastLanguageModel, FastModel

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN    = 768

policy, tokenizer = FastModel.from_pretrained(BASE_MODEL, max_seq_length=MAX_LEN, load_in_4bit=True, full_finetuning=False)
ref_policy, _     = FastModel.from_pretrained(BASE_MODEL, max_seq_length=MAX_LEN, load_in_4bit=True, full_finetuning=False)

# pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

# add LoRA
policy = FastLanguageModel.get_peft_model(
    policy,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16, lora_dropout=0.05, bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)

# 2) Make BOTH models return hidden states; freeze ref; disable cache for training
for m in (policy, ref_policy):
    m.config.output_hidden_states = True
    m.config.use_cache = False
policy.train(); ref_policy.eval()
for p in ref_policy.parameters():
    p.requires_grad = False


==((====))==  Unsloth 2025.9.5: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.9.5: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
# 3) GRPO config with explicit num_generations (batch size must be a multiple)
from trl import GRPOTrainer, GRPOConfig

NUM_GENERATIONS = 2       # try 2 on T4 to reduce memory
PER_DEVICE_BSZ  = 4       # MUST be 2, 4, 6… (multiple of NUM_GENERATIONS)

cfg = GRPOConfig(
    output_dir="outputs_grpo_unsloth",
    per_device_train_batch_size=PER_DEVICE_BSZ,
    gradient_accumulation_steps=1,
    max_steps=100,
    logging_steps=5,
    save_steps=50,
    learning_rate=5e-6,
    max_prompt_length=320,
    max_completion_length=128,
    beta=0.03,
    num_generations=NUM_GENERATIONS,
    dataloader_num_workers=0,
)


In [ ]:
# Must be set BEFORE importing unsloth
import os
os.environ["UNSLOTH_DONT_PATCH_TRAINERS"] = "1"

# Import TRL/Transformers/Torch FIRST
import torch, transformers, trl

# Grab stable references to vanilla TRL classes NOW
VanillaGRPOTrainer = trl.GRPOTrainer
VanillaGRPOConfig  = trl.GRPOConfig

print("Vanilla GRPOTrainer from:", VanillaGRPOTrainer.__module__)

# Now import Unsloth (model speedups), AFTER we saved the vanilla classes
import unsloth
from unsloth import FastLanguageModel, FastModel

print("Torch:", torch.__version__, "| Transformers:", transformers.__version__, "| TRL:", trl.__version__)


Vanilla GRPOTrainer from: UnslothGRPOTrainer
Torch: 2.8.0+cu126 | Transformers: 4.57.1 | TRL: 0.24.0


In [ ]:
!pip -q install "transformers==4.57.1" "trl==0.24.0" \
                "accelerate>=0.34.2" "peft>=0.13.2" \
                "datasets>=2.20.0" "bitsandbytes>=0.45.0" \
                unsloth unsloth_zoo


In [ ]:
# MUST be set before importing unsloth
%env UNSLOTH_DONT_PATCH_TRAINERS=1
import os
os.environ["UNSLOTH_DONT_PATCH_TRAINERS"] = "1"

# Import TRL first, and take references to its vanilla GRPO classes
import torch, transformers, trl
VanillaGRPOTrainer = trl.GRPOTrainer
VanillaGRPOConfig  = trl.GRPOConfig
print("Vanilla GRPOTrainer from:", VanillaGRPOTrainer.__module__)

# Now import Unsloth (model speedups); it will NOT patch trainers
import unsloth
from unsloth import FastLanguageModel, FastModel

print("Torch:", torch.__version__, "| Transformers:", transformers.__version__, "| TRL:", trl.__version__)


env: UNSLOTH_DONT_PATCH_TRAINERS=1


Vanilla GRPOTrainer from: trl.trainer.grpo_trainer


/tmp/ipython-input-2025343976.py:13: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Torch: 2.8.0+cu126 | Transformers: 4.57.1 | TRL: 0.24.0


In [ ]:
# Data + reward (same as before)
from datasets import load_dataset, Dataset
import re

gsm = load_dataset("openai/gsm8k", "main", split="train[:1000]")
def to_prompt(rec):
    return f"""Solve the math word problem step by step, then answer with '#### <final_number>' on the last line.

Problem:
{rec['question']}
"""
prompts    = [to_prompt(x) for x in gsm]
references = [x["answer"] for x in gsm]
train_ds   = Dataset.from_dict({"prompt": prompts})
prompt2gold = dict(zip(prompts, references))

def extract_final_number(s: str):
    m = re.search(r"####\s*(-?\d+(?:\.\d+)?)", s)
    return m.group(1).strip() if m else None

def reward_fn(*, prompts=None, completions=None, **kwargs):
    rewards = []
    for p, gen in zip(prompts or [], completions or []):
        gold = prompt2gold.get(p, "")
        got  = extract_final_number(gen or "")
        ans  = extract_final_number(gold or "")
        rewards.append(1.0 if (ans is not None and got == ans) else 0.0)
    return rewards


In [ ]:
# Unsloth model (4-bit + LoRA)
BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN    = 768

policy, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = True,
    full_finetuning = False,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

policy = FastLanguageModel.get_peft_model(
    policy,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16, lora_dropout=0.05, bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)


==((====))==  Unsloth 2025.9.5: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
# VANILLA TRL GRPO config — T4 safe & divisible
cfg = VanillaGRPOConfig(
    output_dir="outputs_grpo_vanilla",
    per_device_train_batch_size=2,   # small to fit T4
    generation_batch_size=2,         # <-- divisible by num_generations
    num_generations=2,               # <-- lower from default 8

    gradient_accumulation_steps=1,
    max_steps=100,
    logging_steps=5,
    save_steps=50,
    learning_rate=5e-6,

    # keep lengths modest for memory
    max_prompt_length=320,
    max_completion_length=128,

    beta=0.03,
    dataloader_num_workers=0,

    # precision for T4
    bf16=False,
    fp16=True,
    tf32=False,
)

# ✅ Correct for trl==0.24.0 (no tokenizer arg)
trainer = VanillaGRPOTrainer(
    model=policy,
    args=cfg,
    reward_funcs=[reward_fn],
    train_dataset=train_ds,
)


print("✅ Starting GRPO (vanilla TRL with Unsloth model)…")
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


✅ Starting GRPO (vanilla TRL with Unsloth model)…


wandb: Currently logged in as: anuradhasrivastav25 (anuradhasrivastav25-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
5,0.000000
10,0.000000
15,0.000000
20,0.000000
25,0.000000
30,0.000000
35,0.000000
40,0.000000
45,0.000000
50,0.000000


TrainOutput(global_step=100, training_loss=0.0, metrics={'train_runtime': 1956.6608, 'train_samples_per_second': 0.102, 'train_steps_per_second': 0.051, 'total_flos': 0.0, 'train_loss': 0.0})

In [ ]:
print("✅ Starting GRPO (vanilla TRL with Unsloth model)…")
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


✅ Starting GRPO (vanilla TRL with Unsloth model)…


Step,Training Loss
5,0.000000
10,0.000000
15,0.000000
20,0.000000
25,0.000000
30,0.000000
35,0.000000
40,0.000000
45,0.000000
50,0.000000


Step,Training Loss
5,0.000000
10,0.000000
15,0.000000
20,0.000000
25,0.000000
30,0.000000
35,0.000000
40,0.000000
45,0.000000
50,0.000000


TrainOutput(global_step=100, training_loss=0.0, metrics={'train_runtime': 1923.6051, 'train_samples_per_second': 0.104, 'train_steps_per_second': 0.052, 'total_flos': 0.0, 'train_loss': 0.0})

In [ ]:
# ⚡ Fast, batched evaluation on a smaller slice
from datasets import load_dataset
from math import ceil
from tqdm.auto import tqdm
import torch, re, time

# --- knobs you can tweak ---
N_TEST = 80          # try 40–100; smaller = faster
BATCH  = 8           # 4–16 depending on VRAM
MAX_PROMPT_LEN = 320
MAX_NEW = 96         # shorter = faster; 64–128 is fine
USE_FP16 = True      # keep True on T4

# --- helpers ---
def to_prompt(rec):
    return f"""Solve the math word problem step by step, then answer with '#### <final_number>' on the last line.

Problem:
{rec['question']}
"""

def extract_final_number(s: str):
    m = re.search(r"####\s*(-?\d+(?:\.\d+)?)", s)
    return m.group(1).strip() if m else None

# --- data ---
test = load_dataset("openai/gsm8k", "main", split=f"test[:{N_TEST}]")
prompts_eval = [to_prompt(x) for x in test]
answers_eval = [x["answer"] for x in test]

policy.eval()
device = next(policy.parameters()).device

# batch tokenization helper (pad to same length to keep it fast)
def batch_encode(texts):
    return tokenizer(
        texts, return_tensors="pt", padding=True, truncation=True,
        max_length=MAX_PROMPT_LEN
    ).to(device)

correct = 0
gens = []
t0 = time.time()

with torch.inference_mode():
    if USE_FP16 and torch.cuda.is_available():
        autocast_ctx = torch.cuda.amp.autocast(dtype=torch.float16)
    else:
        # No-op context manager
        class _noop:
            def __enter__(self): return None
            def __exit__(self, *a): return False
        autocast_ctx = _noop()

    with autocast_ctx:
        for i in tqdm(range(0, len(prompts_eval), BATCH), desc="Evaluating"):
            batch_prompts = prompts_eval[i:i+BATCH]
            batch_answers = answers_eval[i:i+BATCH]

            x = batch_encode(batch_prompts)

            # greedy decoding is fastest & stable for exact-match metric
            y = policy.generate(
                **x,
                max_new_tokens=MAX_NEW,
                do_sample=False,
                num_beams=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )

            outs = tokenizer.batch_decode(y, skip_special_tokens=True)
            gens.extend(outs)

            for out, gold in zip(outs, batch_answers):
                got = extract_final_number(out)
                ref = extract_final_number(gold)
                if got is not None and got == ref:
                    correct += 1

dt = time.time() - t0
acc = correct / len(prompts_eval)
print(f"\n✅ Exact final-number accuracy: {acc:.3f} on {len(prompts_eval)} problems")
print(f"⏱️ Time: {dt:.1f}s | Throughput: {len(prompts_eval)/dt:.2f} samples/sec")

# peek at a few generations
for k in range(min(3, len(gens))):
    print(f"\n--- Example {k+1} ---")
    print(gens[k][:800])


/tmp/ipython-input-2963814252.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  autocast_ctx = torch.cuda.amp.autocast(dtype=torch.float16)


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le


✅ Exact final-number accuracy: 0.000 on 80 problems
⏱️ Time: 214.9s | Throughput: 0.37 samples/sec

--- Example 1 ---
Solve the math word problem step by step, then answer with '#### <final_number>' on the last line.

Problem:
Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Step 1:

Step 2:

Step 3:

Step 4:

Step 5:

Step 6:

Step 7:

Step 8:

Step 9:

Step 10:

Step 11:

Step 12:

Step 13:

Step 14:

Step 15:


--- Example 2 ---
Solve the math word problem step by step, then answer with '#### <final_number>' on the last line.

Problem:
A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?

Step 1:

Step 2:

Step 3:

Step 4:

Step 5:

Step 6:

Step 7:

Step 8:

Step 9:

Step 10:

Step 11:

Step 12:

Ste

In [ ]:
import re, pandas as pd, os

def extract_final_number(s: str):
    m = re.search(r"####\s*(-?\d+(?:\.\d+)?)", s)
    return m.group(1).strip() if m else None

rows = []
for p, gold, gen in zip(prompts_eval, answers_eval, gens):
    pred = extract_final_number(gen or "")
    ref  = extract_final_number(gold or "")
    rows.append({
        "prompt": p,
        "gold_answer": gold,
        "generated": gen,
        "pred_final_number": pred,
        "ref_final_number": ref,
        "correct": (pred is not None and ref is not None and pred == ref),
    })

df = pd.DataFrame(rows)
csv_path = "grpo_eval_results.csv"
df.to_csv(csv_path, index=False)
print("Saved:", os.path.abspath(csv_path))
df.head(3)


Saved: /content/grpo_eval_results.csv


,prompt,gold_answer,generated,pred_final_number,ref_final_number,correct
0,"Solve the math word problem step by step, then...",Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,"Solve the math word problem step by step, then...",None,18,False
1,"Solve the math word problem step by step, then...",It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,"Solve the math word problem step by step, then...",None,3,False
2,"Solve the math word problem step by step, then...",The cost of the house and repairs came out to ...,"Solve the math word problem step by step, then...",None,70000,False


In [ ]:
import shutil, os

artifacts = [
    "grpo_smollm2_lora_adapters",  # comment out if you didn't save adapters
    "grpo_smollm2_merged_fp16",    # comment out if you didn't merge
    "outputs_grpo_vanilla",        # trainer logs/checkpoints
    "grpo_eval_results.csv",
]

for a in artifacts:
    if os.path.exists(a):
        out = a.rstrip("/").replace("/", "_") + ".zip"
        shutil.make_archive(a, 'zip', a) if os.path.isdir(a) else shutil.copy(a, out)
        print("Zipped:", out)
    else:
        print("Skip (not found):", a)


Skip (not found): grpo_smollm2_lora_adapters
Skip (not found): grpo_smollm2_merged_fp16
Zipped: outputs_grpo_vanilla.zip
Zipped: grpo_eval_results.csv.zip


In [ ]:
SAVE_DIR = "grpo_smollm2_final"
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("Saved to:", SAVE_DIR)


Saved to: grpo_smollm2_final


# Continued Pretraining

In [ ]:
!pip -q install "transformers==4.57.1" "peft==0.13.2" \
                "datasets>=2.20.0" "accelerate>=0.34.2" \
                "bitsandbytes>=0.45.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.0 MB/s eta 0:00:00


In [ ]:
try:
    import unsloth
    raise RuntimeError("unsloth is present (should NOT be). Do: Runtime → Factory reset runtime, then rerun installs.")
except Exception as e:
    print("✅ unsloth not importable (good):", type(e).__name__)


✅ unsloth not importable (good): ModuleNotFoundError


In [ ]:
# ✅ Build Hindi text dataset (streamed) → train_ds / valid_ds
from datasets import load_dataset, Dataset
from itertools import islice

N_LINES = 8000   # lower to 6000 if you want it faster

def get_hi_stream():
    # 1) Parquet-backed Wikipedia from the wikimedia org (no custom code)
    for snap in ["20231101.hi", "20240101.hi"]:
        try:
            print(f"Trying wikimedia/wikipedia:{snap} (parquet)…")
            return load_dataset("wikimedia/wikipedia", snap, split="train", streaming=True)
        except Exception as e:
            print("  -> failed:", type(e).__name__)
    # 2) Classic wikipedia loader (requires trust_remote_code)
    for snap in ["20231101.hi", "20230601.hi"]:
        try:
            print(f"Trying wikipedia:{snap} with trust_remote_code…")
            return load_dataset("wikipedia", snap, split="train", streaming=True, trust_remote_code=True)
        except Exception as e:
            print("  -> failed:", type(e).__name__)
    raise RuntimeError("No public Hindi Wikipedia snapshot available in this runtime.")

stream = get_hi_stream()

def cleaned_lines(gen):
    for row in gen:
        txt = (row.get("text") or "").strip()
        if len(txt) > 10:
            yield txt

sample = list(islice(cleaned_lines(stream), N_LINES))
if len(sample) < 1000:
    raise RuntimeError(f"Only {len(sample)} lines collected — increase N_LINES or re-run.")

cut = int(len(sample) * 0.98)
train_ds = Dataset.from_dict({"text": sample[:cut]})
valid_ds = Dataset.from_dict({"text": sample[cut:]})

print(f"✅ Dataset ready: {len(train_ds)} train / {len(valid_ds)} valid")
print("• Example:", train_ds[0]["text"][:140].replace("\n"," "))


Trying wikimedia/wikipedia:20231101.hi (parquet)…


README.md: 0.00B [00:00, ?B/s]

✅ Dataset ready: 7840 train / 160 valid
• Example: हम होंगे कामयाब ( का गिरिजा कुमार माथुर द्वारा किया गया हिंदी भावानुवाद) एक प्रतिरोध गीत है। यह गीत बीसवीं सदी में नागरिक अधिकार आंदोलन का प


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN    = 512

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.model_max_length = MAX_LEN

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, quantization_config=bnb_cfg, device_map="auto",
)
model.config.use_cache = False

lora_cfg = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

trainable params: 4,884,480 || all params: 139,400,064 || trainable%: 3.5039


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

def tok_fn(batch):
    out = tokenizer(
        batch["text"], truncation=True, max_length=MAX_LEN, padding="max_length",
    )
    out["labels"] = out["input_ids"].copy()
    return out

cols = list(train_ds.column_names)
t_train = train_ds.map(tok_fn, batched=True, remove_columns=cols)
t_eval  = valid_ds.map(tok_fn, batched=True, remove_columns=cols)
t_train.set_format("torch"); t_eval.set_format("torch")

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

args = TrainingArguments(
    output_dir="outputs_cpt_hi_hf",
    per_device_train_batch_size=4,     # if OOM: 2
    gradient_accumulation_steps=2,     # effective batch ~8
    num_train_epochs=1,
    learning_rate=2e-4, warmup_ratio=0.03, weight_decay=0.0,
    logging_steps=20, save_steps=200, save_total_limit=2,
    fp16=True, bf16=False, tf32=False, report_to=[], dataloader_num_workers=0,
)

trainer = Trainer(
    model=model, args=args,
    train_dataset=t_train, eval_dataset=t_eval,
    data_collator=collator,
)

print("✅ Starting Continued Pretraining (Hindi)…")
trainer.train()


Map:   0%|          | 0/7840 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

✅ Starting Continued Pretraining (Hindi)…


Step,Training Loss
20,1.874500
40,1.730400
60,1.579600
80,1.576900
100,1.505600
120,1.400600
140,1.481300
160,1.382700
180,1.343600
200,1.344900


Step,Training Loss
20,1.874500
40,1.730400
60,1.579600
80,1.576900
100,1.505600
120,1.400600
140,1.481300
160,1.382700
180,1.343600
200,1.344900


In [ ]:
import math, os
metrics = trainer.evaluate()
loss = metrics.get("eval_loss", None)
ppl = math.exp(loss) if loss is not None else None
print(f"Validation loss: {loss} | Perplexity: {ppl}")

out_dir = "cpt_hi_smollm2_lora_adapters"
model.save_pretrained(out_dir)
tokenizer.save_pretrained(out_dir)
print("💾 Saved adapters to:", os.path.abspath(out_dir))


Validation loss: 1.1026958227157593 | Perplexity: 3.012275649100189
💾 Saved adapters to: /content/cpt_hi_smollm2_lora_adapters


In [ ]:
prompt = "भारत की स्वतंत्रता के इतिहास पर तीन वाक्य लिखिए।"
x = tokenizer(prompt, return_tensors="pt").to(next(model.parameters()).device)
with torch.inference_mode():
    y = model.generate(**x, max_new_tokens=120, do_sample=True, temperature=0.8, top_p=0.9)
print(tokenizer.decode(y[0], skip_special_tokens=True))


भारत की स्वतंत्रता के इतिहास पर तीन वाक्य लिखिए। यह विद्यालय जिसके दक्षिणी व्यापारिक वाहवंश पर आर्षिक तीन वाक्य लिखिए। वेला महान ने उपनायक में वाक्य लिखि


In [ ]:
import os, shutil

ARTIFACTS = [
    "cpt_hi_smollm2_lora_adapters",  # adapters folder from step 6
    "outputs_cpt_hi_hf",             # trainer logs/checkpoints
]

for path in ARTIFACTS:
    if os.path.exists(path):
        out = path.rstrip("/").replace("/", "_") + ".zip"
        if os.path.isdir(path):
            shutil.make_archive(path, 'zip', path)
            print("Zipped:", out)
        else:
            shutil.copy(path, out)
            print("Copied:", out)
    else:
        print("Skip (not found):", path)


Zipped: cpt_hi_smollm2_lora_adapters.zip
Zipped: outputs_cpt_hi_hf.zip


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "unsloth/smollm2-135m"
MERGED_DIR = "cpt_hi_smollm2_merged_fp16"
ADAPTERS   = "cpt_hi_smollm2_lora_adapters"

# reload base in fp16 (not 4-bit) → clean merge
base_fp16 = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, torch_dtype=torch.float16, device_map="auto"
)
merged = PeftModel.from_pretrained(base_fp16, ADAPTERS).merge_and_unload()

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
merged.save_pretrained(MERGED_DIR)
tok.save_pretrained(MERGED_DIR)
print("✅ Merged model saved to:", MERGED_DIR)


`torch_dtype` is deprecated! Use `dtype` instead!


✅ Merged model saved to: cpt_hi_smollm2_merged_fp16


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Use adapters model (PEFT) – or switch to MERGED_DIR if you merged
from peft import PeftModel
BASE_MODEL = "unsloth/smollm2-135m"
ADAPTERS   = "cpt_hi_smollm2_lora_adapters"

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, device_map="auto", torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base, ADAPTERS)

prompt = "भारत की स्वतंत्रता के इतिहास पर तीन वाक्य लिखिए।"
x = tok(prompt, return_tensors="pt").to(model.device)
with torch.inference_mode():
    y = model.generate(**x, max_new_tokens=120, do_sample=True, temperature=0.8, top_p=0.9)
print(tok.decode(y[0], skip_special_tokens=True))


भारत की स्वतंत्रता के इतिहास पर तीन वाक्य लिखिए।

किन्तु प्रदान 
उदाहरण प्राचीन में बृहण 1895-1943, प्राचीन उदाहरण प्राचीन विज्ञान में से 1933-1933 में एक उ


In [ ]:
# Save continued-pretraining model on Hindi corpus
SAVE_DIR = "continued_pretraining_hindi_smollm2_final"

trainer.save_model(SAVE_DIR)           # saves model weights + config
tokenizer.save_pretrained(SAVE_DIR)    # saves tokenizer files
print("Saved to:", SAVE_DIR)


Saved to: continued_pretraining_hindi_smollm2_final


# DPO (Direct Preference Optimization)

In [ ]:
!pip -q install "transformers==4.57.1" "trl==0.24.0" \
                "peft==0.13.2" "accelerate>=0.34.2" \
                "datasets>=2.20.0" "bitsandbytes>=0.45.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 32.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = "unsloth/smollm2-135m"
MAX_PROMPT_LEN = 256
MAX_COMPLETION_LEN = 256
MAX_LEN = MAX_PROMPT_LEN + MAX_COMPLETION_LEN

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.model_max_length = MAX_LEN

# try 4-bit; fall back to fp16 if bitsandbytes is unavailable
def load_model():
    try:
        from transformers import BitsAndBytesConfig
        bnb_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
        )
        print("✅ Loading 4-bit quantized model…")
        m = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL,
            quantization_config=bnb_cfg,
            device_map="auto",
        )
    except Exception as e:
        print("⚠️ 4-bit load failed, falling back to fp16:", type(e).__name__, "-", e)
        m = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL,
            torch_dtype=torch.float16,
            device_map="auto",
        )
    m.config.use_cache = False
    return m

model = load_model()
print("Model device:", next(model.parameters()).device)


✅ Loading 4-bit quantized model…


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


⚠️ 4-bit load failed, falling back to fp16: ImportError - Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

Model device: cuda:0


In [ ]:
# --- force PEFT to not use bitsandbytes ---
import os
os.environ["BNB_TRITON_DISABLE"] = "1"   # harmless even if bnb isn't present

# If bnb was ever partially imported this session, nuke it:
import sys
for k in list(sys.modules.keys()):
    if k.startswith("bitsandbytes"):
        sys.modules.pop(k, None)

# Monkeypatch PEFT's bnb detection to always False
import peft
import peft.tuners.lora.model as lora_model
lora_model.is_bnb_available = lambda: False

print("✅ Patched PEFT: is_bnb_available() -> False")


✅ Patched PEFT: is_bnb_available() -> False


In [ ]:
from peft import LoraConfig, get_peft_model

lora_cfg = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_cfg)   # <-- will not import bitsandbytes now
model.print_trainable_parameters()


trainable params: 4,884,480 || all params: 139,400,064 || trainable%: 3.5039


In [ ]:
from datasets import load_dataset

# Small, fast slice for demo; increase later (e.g., train[:8000])
raw = load_dataset("Anthropic/hh-rlhf", split="train[:2000]")

def add_prompt(row):
    ch = row["chosen"] or ""
    first_line = ch.split("\n", 1)[0].strip()
    row["prompt"] = first_line[:200] if first_line else "Please respond helpfully."
    return row

ds = raw.map(add_prompt, desc="Adding prompt")
ds = ds.select_columns(["prompt", "chosen", "rejected"])
print(ds[0])
print("✅ Dataset ready:", ds)


README.md: 0.00B [00:00, ?B/s]

harmless-base/train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

helpful-base/train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

helpful-online/train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

helpful-rejection-sampled/train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

helpful-rejection-sampled/test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

Adding prompt:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'prompt': 'Please respond helpfully.', 'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.", 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-suckin

In [ ]:
!pip install -q "trl==0.8.6" "transformers==4.44.2" "accelerate" "datasets" "peft==0.12.0"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.1 MB/s eta 0:00:00


In [ ]:
import trl
print("TRL version:", trl.__version__)


TRL version: 0.8.6


In [ ]:
from transformers import AutoTokenizer

BASE_MODEL = "unsloth/smollm2-135m"

# Force the Python (slow) tokenizer — avoids the Rust backend entirely
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    use_fast=False,          # 👈 key change
)
print("Loaded tokenizer (slow). pad_token:", tokenizer.pad_token)


Loaded tokenizer (slow). pad_token: <|PAD_TOKEN|>


In [ ]:
# install a compatible tokenizers for transformers 4.57.1
!pip -q install --no-deps "tokenizers==0.20.3"

# sanity: show versions
import importlib.metadata as im
for p in ["transformers","tokenizers","trl","accelerate","peft","datasets"]:
    try: print(p, im.version(p))
    except: print(p, "❌ NOT INSTALLED")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.9 MB/s eta 0:00:00
transformers 4.57.1
tokenizers 0.20.3
trl 0.24.0
accelerate 0.33.0
peft 0.13.2
datasets 4.0.0


In [ ]:
!pip -q install --no-deps --force-reinstall "tokenizers==0.22.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 512

# Try fast; if it errors, switch to use_fast=False
try:
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
except Exception:
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.model_max_length = MAX_LEN

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, torch_dtype=torch.float16, device_map="auto"
)
model.config.use_cache = False
print("✅ Loaded. Device:", next(model.parameters()).device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


✅ Loaded. Device: cuda:0


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = "unsloth/smollm2-135m"
MAX_PROMPT_LEN = 256
MAX_COMPLETION_LEN = 256
MAX_LEN = MAX_PROMPT_LEN + MAX_COMPLETION_LEN

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.model_max_length = MAX_LEN

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, torch_dtype=torch.float16, device_map="auto"
)
model.config.use_cache = False
print("✅ Loaded. Device:", next(model.parameters()).device)


✅ Loaded. Device: cuda:0


In [ ]:
# Block any accidental bitsandbytes usage
import os, sys
for k in list(sys.modules.keys()):
    if k.startswith("bitsandbytes"):
        sys.modules.pop(k, None)
os.environ["BNB_TRITON_DISABLE"] = "1"

import peft
import peft.import_utils as peft_import_utils
peft_import_utils.is_bnb_available = lambda: False
peft_import_utils.is_bnb_4bit_available = lambda: False
try:
    import peft.tuners.lora.model as lora_model
    lora_model.is_bnb_available = lambda: False
    lora_model.is_bnb_4bit_available = lambda: False
except Exception:
    pass
print("PEFT bnb checks:", peft_import_utils.is_bnb_available(), peft_import_utils.is_bnb_4bit_available())

from peft import LoraConfig, get_peft_model
lora_cfg = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()


PEFT bnb checks: False False
trainable params: 4,884,480 || all params: 139,400,064 || trainable%: 3.5039


In [ ]:
from datasets import load_dataset

raw = load_dataset("Anthropic/hh-rlhf", split="train[:2000]")

def add_prompt(row):
    ch = row["chosen"] or ""
    first = ch.split("\n", 1)[0].strip()
    row["prompt"] = first[:200] if first else "Please respond helpfully."
    return row

ds = raw.map(add_prompt, desc="Adding prompt").select_columns(["prompt","chosen","rejected"])
print(ds[0]); print("✅ Dataset size:", len(ds))


Adding prompt:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'prompt': 'Please respond helpfully.', 'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.", 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-suckin

In [ ]:
!pip -q install --no-deps "accelerate==0.34.2"
import os; os.kill(os.getpid(), 9)


In [ ]:
import importlib.metadata as im
for p in ["transformers","tokenizers","trl","accelerate","peft","datasets"]:
    try: print(p, im.version(p))
    except: print(p, "❌ NOT INSTALLED")
# Expect: accelerate 0.34.2 (with transformers 4.57.1, tokenizers 0.22.x, trl 0.24.0)


transformers 4.57.1
tokenizers 0.22.0
trl 0.24.0
accelerate 0.34.2
peft 0.13.2
datasets 4.0.0


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = "unsloth/smollm2-135m"
MAX_PROMPT_LEN = 256
MAX_COMPLETION_LEN = 256
MAX_LEN = MAX_PROMPT_LEN + MAX_COMPLETION_LEN

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.model_max_length = MAX_LEN

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, torch_dtype=torch.float16, device_map="auto"
)
model.config.use_cache = False
print("✅ Loaded. Device:", next(model.parameters()).device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


✅ Loaded. Device: cuda:0


In [ ]:
# Block any accidental bitsandbytes usage
import os, sys
for k in list(sys.modules.keys()):
    if k.startswith("bitsandbytes"):
        sys.modules.pop(k, None)
os.environ["BNB_TRITON_DISABLE"] = "1"

import peft
import peft.import_utils as peft_import_utils
peft_import_utils.is_bnb_available = lambda: False
peft_import_utils.is_bnb_4bit_available = lambda: False
try:
    import peft.tuners.lora.model as lora_model
    lora_model.is_bnb_available = lambda: False
    lora_model.is_bnb_4bit_available = lambda: False
except Exception:
    pass
print("PEFT bnb checks:", peft_import_utils.is_bnb_available(), peft_import_utils.is_bnb_4bit_available())

from peft import LoraConfig, get_peft_model
lora_cfg = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()


PEFT bnb checks: False False
trainable params: 4,884,480 || all params: 139,400,064 || trainable%: 3.5039


In [ ]:
from datasets import load_dataset

raw = load_dataset("Anthropic/hh-rlhf", split="train[:2000]")

def add_prompt(row):
    ch = row["chosen"] or ""
    first = ch.split("\n", 1)[0].strip()
    row["prompt"] = first[:200] if first else "Please respond helpfully."
    return row

ds = raw.map(add_prompt, desc="Adding prompt").select_columns(["prompt","chosen","rejected"])
print(ds[0]); print("✅ Dataset size:", len(ds))


{'prompt': 'Please respond helpfully.', 'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.", 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-suckin

In [ ]:
# Keep the earlier ds (columns: prompt/chosen/rejected) and tokenizer
MAX_PROMPT_LEN = 256
MAX_COMPLETION_LEN = 256

def _truncate_to(text, max_len):
    ids = tokenizer(
        text,
        truncation=True,
        max_length=max_len,
        add_special_tokens=False,
    )["input_ids"]
    return tokenizer.decode(ids, skip_special_tokens=True)

def clip_completions(row):
    row["chosen"]   = _truncate_to(row["chosen"],   MAX_COMPLETION_LEN)
    row["rejected"] = _truncate_to(row["rejected"], MAX_COMPLETION_LEN)
    return row

ds = ds.map(clip_completions, desc="Truncating chosen/rejected")


Truncating chosen/rejected:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
MAX_PROMPT_LEN = 256
MAX_COMPLETION_LEN = 256

def _truncate_to(text, max_len):
    ids = tokenizer(text, truncation=True, max_length=max_len, add_special_tokens=False)["input_ids"]
    return tokenizer.decode(ids, skip_special_tokens=True)

def clip_all(row):
    row["prompt"]   = _truncate_to(row["prompt"],   MAX_PROMPT_LEN)
    row["chosen"]   = _truncate_to(row["chosen"],   MAX_COMPLETION_LEN)
    row["rejected"] = _truncate_to(row["rejected"], MAX_COMPLETION_LEN)
    return row

ds = ds.map(clip_all, desc="Clipping prompt/chosen/rejected")


Clipping prompt/chosen/rejected:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
def lens(row):
    return {
        "prompt_len":   len(tokenizer(row["prompt"], add_special_tokens=False)["input_ids"]),
        "chosen_len":   len(tokenizer(row["chosen"], add_special_tokens=False)["input_ids"]),
        "rejected_len": len(tokenizer(row["rejected"], add_special_tokens=False)["input_ids"]),
    }

sample = ds.select(range(32)).map(lens)
max_prompt   = max(sample["prompt_len"])
max_chosen   = max(sample["chosen_len"])
max_rejected = max(sample["rejected_len"])
print("Max prompt/chosen/rejected:", max_prompt, max_chosen, max_rejected)
print("Model max:", tokenizer.model_max_length)


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Max prompt/chosen/rejected: 5 256 256
Model max: 512


In [ ]:
!pip -q install --no-deps "accelerate==0.34.2"
import os; os.kill(os.getpid(), 9)


In [ ]:
import importlib.metadata as im
for p in ["transformers","tokenizers","trl","accelerate","peft","datasets"]:
    try: print(p, im.version(p))
    except: print(p, "❌ NOT INSTALLED")
# Expect: transformers 4.57.1 | tokenizers 0.22.x | trl 0.24.0 | accelerate 0.34.2 | peft 0.13.2


transformers 4.57.1
tokenizers 0.22.0
trl 0.24.0
accelerate 0.34.2
peft 0.13.2
datasets 4.0.0


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = "unsloth/smollm2-135m"
MAX_PROMPT_LEN, MAX_COMPLETION_LEN = 256, 256
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.model_max_length = MAX_PROMPT_LEN + MAX_COMPLETION_LEN

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, torch_dtype=torch.float16, device_map="auto"
)
model.config.use_cache = False
print("✅ Loaded. Device:", next(model.parameters()).device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


✅ Loaded. Device: cuda:0


In [ ]:
import os, sys
for k in list(sys.modules.keys()):
    if k.startswith("bitsandbytes"):
        sys.modules.pop(k, None)
os.environ["BNB_TRITON_DISABLE"] = "1"

import peft
import peft.import_utils as peft_import_utils
peft_import_utils.is_bnb_available = lambda: False
peft_import_utils.is_bnb_4bit_available = lambda: False
try:
    import peft.tuners.lora.model as lora_model
    lora_model.is_bnb_available = lambda: False
    lora_model.is_bnb_4bit_available = lambda: False
except Exception:
    pass

from peft import LoraConfig, get_peft_model
lora_cfg = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()


trainable params: 4,884,480 || all params: 139,400,064 || trainable%: 3.5039


In [ ]:
from datasets import load_dataset

raw = load_dataset("Anthropic/hh-rlhf", split="train[:2000]")

def add_prompt(row):
    ch = row["chosen"] or ""
    first = ch.split("\n", 1)[0].strip()
    row["prompt"] = first[:200] if first else "Please respond helpfully."
    return row

ds = raw.map(add_prompt, desc="Adding prompt").select_columns(["prompt","chosen","rejected"])

def _truncate_to(text, max_len):
    ids = tokenizer(text, truncation=True, max_length=max_len, add_special_tokens=False)["input_ids"]
    return tokenizer.decode(ids, skip_special_tokens=True)

def clip_all(row):
    row["prompt"]   = _truncate_to(row["prompt"],   MAX_PROMPT_LEN)
    row["chosen"]   = _truncate_to(row["chosen"],   MAX_COMPLETION_LEN)
    row["rejected"] = _truncate_to(row["rejected"], MAX_COMPLETION_LEN)
    return row

ds = ds.map(clip_all, desc="Clipping prompt/chosen/rejected")
print(ds[0]); print("✅ Dataset size:", len(ds))


Adding prompt:   0%|          | 0/2000 [00:00<?, ? examples/s]

Clipping prompt/chosen/rejected:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'prompt': 'Please respond helpfully.', 'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.", 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-suckin

In [ ]:
# Make accelerate compatible with transformers 4.57.1
!pip -q install --no-deps --force-reinstall "accelerate==0.34.2"

# Hard-restart Python so the new wheel is actually used
import os; os.kill(os.getpid(), 9)


In [ ]:
import importlib.metadata as im
for p in ["transformers","tokenizers","trl","accelerate","peft","datasets"]:
    try: print(p, im.version(p))
    except: print(p, "❌ NOT INSTALLED")
# Expect: transformers 4.57.1 | tokenizers 0.22.x | trl 0.24.0 | accelerate 0.34.2 | peft 0.13.2


transformers 4.57.1
tokenizers 0.22.0
trl 0.24.0
accelerate 0.34.2
peft 0.13.2
datasets 4.0.0


In [ ]:
# Define these again before your DPOConfig
MAX_PROMPT_LEN = 256
MAX_COMPLETION_LEN = 256

from trl import DPOConfig

dpo_cfg = DPOConfig(
    output_dir="outputs_dpo_smollm2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    max_steps=200,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    learning_rate=5e-6,
    beta=0.1,  # Goes inside config
    max_prompt_length=MAX_PROMPT_LEN,
    max_completion_length=MAX_COMPLETION_LEN,
    fp16=True,
    bf16=False,
    tf32=False,
    report_to=[],
    dataloader_num_workers=0,
    remove_unused_columns=False,
)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

BASE_MODEL = "unsloth/smollm2-135m"
MAX_PROMPT_LEN = 256
MAX_COMPLETION_LEN = 256

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = MAX_PROMPT_LEN + MAX_COMPLETION_LEN

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.config.use_cache = False

print("✅ Model + Tokenizer ready on", next(model.parameters()).device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


✅ Model + Tokenizer ready on cuda:0


In [ ]:
# === Rebuild the tiny preference dataset for DPO ===
from datasets import load_dataset

# If tokenizer isn't in scope (after a restart), quickly reload it
try:
    tokenizer
except NameError:
    from transformers import AutoTokenizer
    BASE_MODEL = "unsloth/smollm2-135m"
    MAX_PROMPT_LEN, MAX_COMPLETION_LEN = 256, 256
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.model_max_length = MAX_PROMPT_LEN + MAX_COMPLETION_LEN

raw = load_dataset("Anthropic/hh-rlhf", split="train[:2000]")

def add_prompt(row):
    ch = row["chosen"] or ""
    first = ch.split("\n", 1)[0].strip()
    row["prompt"] = first[:200] if first else "Please respond helpfully."
    return row

ds = raw.map(add_prompt, desc="Adding prompt").select_columns(["prompt","chosen","rejected"])

def _truncate_to(text, max_len):
    ids = tokenizer(text, truncation=True, max_length=max_len, add_special_tokens=False)["input_ids"]
    return tokenizer.decode(ids, skip_special_tokens=True)

def clip_all(row):
    row["prompt"]   = _truncate_to(row["prompt"],   MAX_PROMPT_LEN)
    row["chosen"]   = _truncate_to(row["chosen"],   MAX_COMPLETION_LEN)
    row["rejected"] = _truncate_to(row["rejected"], MAX_COMPLETION_LEN)
    return row

ds = ds.map(clip_all, desc="Clipping prompt/chosen/rejected")
print("✅ ds ready with columns:", ds.column_names, "| size:", len(ds))
print(ds[0])


Adding prompt:   0%|          | 0/2000 [00:00<?, ? examples/s]

Clipping prompt/chosen/rejected:   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ ds ready with columns: ['prompt', 'chosen', 'rejected'] | size: 2000
{'prompt': 'Please respond helpfully.', 'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.", 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, 

In [ ]:
# === One-shot compatibility patch for accelerate < 0.34 ===
import inspect, accelerate

sig = inspect.signature(accelerate.Accelerator.unwrap_model)
needs_patch = "keep_torch_compile" not in sig.parameters

if needs_patch:
    _old_unwrap = accelerate.Accelerator.unwrap_model
    def _unwrap_compat(self, model, *args, **kwargs):
        kwargs.pop("keep_torch_compile", None)  # drop unknown kwarg
        return _old_unwrap(self, model, *args, **kwargs)
    accelerate.Accelerator.unwrap_model = _unwrap_compat
    print("🩹 Patched accelerate.Accelerator.unwrap_model (no restart needed).")
else:
    print("✅ accelerate already supports keep_torch_compile.")

# (Optional) show versions once so we know the runtime state
try:
    import importlib.metadata as im
    print("versions:",
          "transformers", im.version("transformers"),
          "| tokenizers", im.version("tokenizers"),
          "| trl", im.version("trl"),
          "| accelerate", im.version("accelerate"),
          "| peft", im.version("peft"))
except Exception as _e:
    print("version-check skipped:", _e)


🩹 Patched accelerate.Accelerator.unwrap_model (no restart needed).
versions: transformers 4.57.1 | tokenizers 0.22.0 | trl 0.24.0 | accelerate 0.34.2 | peft 0.13.2


In [ ]:
# --- Minimal collator for TRL DPOTrainer (version-proof) ---
import torch

class SimpleDPOCollator:
    """
    Expects dataset items with keys: 'prompt', 'chosen', 'rejected'.
    Returns Long tensors:
      chosen_input_ids / chosen_attention_mask
      rejected_input_ids / rejected_attention_mask
    """
    def __init__(self, tokenizer, max_len: int):
        self.tok = tokenizer
        self.max_len = max_len

    def __call__(self, features):
        prompts  = [ex["prompt"] for ex in features]
        chosens  = [ex["chosen"] for ex in features]
        rejecteds= [ex["rejected"] for ex in features]

        # Concatenate prompt + completion (chosen/rejected)
        chosen_texts   = [f"{p}\n{c}" for p, c in zip(prompts, chosens)]
        rejected_texts = [f"{p}\n{r}" for p, r in zip(prompts, rejecteds)]

        ch = self.tok(
            chosen_texts,
            padding=True,
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt",
        )
        rj = self.tok(
            rejected_texts,
            padding=True,
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt",
        )

        # ensure Long dtype for embedding lookups
        ch["input_ids"] = ch["input_ids"].long()
        rj["input_ids"] = rj["input_ids"].long()
        ch["attention_mask"] = ch["attention_mask"].long()
        rj["attention_mask"] = rj["attention_mask"].long()

        return {
            "chosen_input_ids":   ch["input_ids"],
            "chosen_attention_mask": ch["attention_mask"],
            "rejected_input_ids": rj["input_ids"],
            "rejected_attention_mask": rj["attention_mask"],
        }

# build the collator
MAX_TOTAL_LEN = tokenizer.model_max_length  # e.g. 512
collator = SimpleDPOCollator(tokenizer, MAX_TOTAL_LEN)
print("✅ SimpleDPOCollator ready (max_len =", MAX_TOTAL_LEN, ")")


✅ SimpleDPOCollator ready (max_len = 512 )


In [ ]:
# Patch unwrap_model only if your accelerate doesn't know the kwarg
import inspect, accelerate
if "keep_torch_compile" not in inspect.signature(accelerate.Accelerator.unwrap_model).parameters:
    _old = accelerate.Accelerator.unwrap_model
    def _unwrap_compat(self, model, *args, **kwargs):
        kwargs.pop("keep_torch_compile", None)
        return _old(self, model, *args, **kwargs)
    accelerate.Accelerator.unwrap_model = _unwrap_compat
    print("🩹 Patched accelerate.Accelerator.unwrap_model")


🩹 Patched accelerate.Accelerator.unwrap_model


In [ ]:
from datasets import load_dataset

# If not already in memory, re-create tokenizer, model_max_length, etc.
try:
    tokenizer
except NameError:
    from transformers import AutoTokenizer
    BASE_MODEL = "unsloth/smollm2-135m"
    MAX_PROMPT_LEN, MAX_COMPLETION_LEN = 256, 256
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.model_max_length = MAX_PROMPT_LEN + MAX_COMPLETION_LEN

raw = load_dataset("Anthropic/hh-rlhf", split="train[:2000]")

def add_prompt(row):
    ch = row["chosen"] or ""
    first = ch.split("\n", 1)[0].strip()
    row["prompt"] = first[:200] if first else "Please respond helpfully."
    return row

ds_text = raw.map(add_prompt).select_columns(["prompt","chosen","rejected"])

def _truncate_to(text, max_len):
    ids = tokenizer(text, truncation=True, max_length=max_len, add_special_tokens=False)["input_ids"]
    return tokenizer.decode(ids, skip_special_tokens=True)

def clip_all(row):
    row["prompt"]   = _truncate_to(row["prompt"],   MAX_PROMPT_LEN)
    row["chosen"]   = _truncate_to(row["chosen"],   MAX_COMPLETION_LEN)
    row["rejected"] = _truncate_to(row["rejected"], MAX_COMPLETION_LEN)
    return row

ds_text = ds_text.map(clip_all, desc="Clipping prompt/chosen/rejected")
print("✅ ds_text columns:", ds_text.column_names, "| size:", len(ds_text))


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Clipping prompt/chosen/rejected:   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ ds_text columns: ['prompt', 'chosen', 'rejected'] | size: 2000


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

BASE_MODEL = "unsloth/smollm2-135m"

# ⬇️ key change: use_fast=False
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False


In [ ]:
from datasets import load_dataset

# Small subset of HH-RLHF dataset
raw = load_dataset("Anthropic/hh-rlhf", split="train[:2000]")

# Add a short "prompt" extracted from the chosen text
def add_prompt(row):
    ch = row["chosen"] or ""
    first = ch.split("\n", 1)[0].strip()
    row["prompt"] = first[:200] if first else "Please respond helpfully."
    return row

ds = raw.map(add_prompt).select_columns(["prompt", "chosen", "rejected"])
print("✅ Dataset columns:", ds.column_names)
print("Example:\n", ds[0])


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Dataset columns: ['prompt', 'chosen', 'rejected']
Example:
 {'prompt': 'Please respond helpfully.', 'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.", 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucke

In [ ]:
import torch

MAX_PROMPT_LEN, MAX_COMPLETION_LEN = 256, 256  # keep same as earlier

def dpo_collator(features):
    # Text lists
    prompts   = [f["prompt"]   for f in features]
    chosens   = [f["chosen"]   for f in features]
    rejecteds = [f["rejected"] for f in features]

    # Tokenize & pad each field separately
    p = tokenizer(
        prompts, padding=True, truncation=True,
        max_length=MAX_PROMPT_LEN, return_tensors="pt"
    )
    c = tokenizer(
        chosens, padding=True, truncation=True,
        max_length=MAX_COMPLETION_LEN, return_tensors="pt"
    )
    r = tokenizer(
        rejecteds, padding=True, truncation=True,
        max_length=MAX_COMPLETION_LEN, return_tensors="pt"
    )

    # Return the exact keys DPOTrainer expects when no tokenizer/processing_class is given
    return {
        "prompt_input_ids":         p["input_ids"],
        "prompt_attention_mask":    p["attention_mask"],
        "chosen_input_ids":         c["input_ids"],
        "chosen_attention_mask":    c["attention_mask"],
        "rejected_input_ids":       r["input_ids"],
        "rejected_attention_mask":  r["attention_mask"],
    }


In [ ]:
import torch
from trl import DPOTrainer
from transformers import TrainingArguments

# --- if you *don't* already have dpo_cfg, uncomment this minimal one ---
# dpo_cfg = TrainingArguments(
#     output_dir="outputs_dpo_smollm2",
#     per_device_train_batch_size=4,   # if OOM: 2
#     gradient_accumulation_steps=1,
#     num_train_epochs=1,
#     logging_steps=10,
#     save_steps=50,
#     learning_rate=5e-6,
#     fp16=False, bf16=False,          # we disable AMP
#     report_to=[],
#     remove_unused_columns=False,
#     optim="adamw_torch",
# )

MAX_PROMPT_LEN = 256
MAX_COMPLETION_LEN = 256

trainer = DPOTrainer(
    model=model,
    args=dpo_cfg,              # your TrainingArguments from before (fp16=False)
    train_dataset=ds,          # columns: prompt / chosen / rejected
    tokenizer=tokenizer,
    max_prompt_length=MAX_PROMPT_LEN,
    max_target_length=MAX_COMPLETION_LEN,
    max_length=MAX_PROMPT_LEN + MAX_COMPLETION_LEN,
    precompute_ref_log_probs=False,
)

# ------------------ HARD DISABLE GradScaler / AMP in Accelerate ------------------
# Some Accelerate/Trainer combos still build a GradScaler. We disable it safely.
acc = trainer.accelerator

# 1) Turn off the scaler if present
if getattr(acc, "scaler", None) is not None:
    try:
        # Preferred: mark scaler as disabled
        acc.scaler._enabled = False
    except Exception:
        # Fallback: remove reference
        acc.scaler = None

# 2) No-op unscale to avoid "Attempting to unscale FP16 gradients."
def _noop_unscale(*args, **kwargs):
    return None
acc.unscale_gradients = _noop_unscale

# 3) Make sure no autocast sneaks in
try:
    torch.set_autocast_enabled(False)  # works on newer torch; harmless otherwise
except Exception:
    pass

# 4) Ensure model/optimizer are in fp32
model.to(torch.float32)

print("🚀 Starting DPO training (AMP force-disabled)…")
trainer.train()


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

🚀 Starting DPO training (AMP force-disabled)…


Step,Training Loss
10,0.692400
20,0.690900
30,0.691900
40,0.684600
50,0.687400
60,0.686900
70,0.675800
80,0.674700
90,0.710300
100,0.659800


TrainOutput(global_step=500, training_loss=0.6702428722381591, metrics={'train_runtime': 579.5888, 'train_samples_per_second': 3.451, 'train_steps_per_second': 0.863, 'total_flos': 0.0, 'train_loss': 0.6702428722381591, 'epoch': 1.0})

In [ ]:
import torch
from transformers import TextStreamer

gen_kwargs = dict(
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.05,
    pad_token_id=tokenizer.eos_token_id,
)

def chat(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(trainer.model.device)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    with torch.no_grad():
        out = trainer.model.generate(**inputs, streamer=streamer, **gen_kwargs)
    print("\n")

test_prompts = [
    "Explain what DPO (Direct Preference Optimization) is in 2-3 sentences.",
    "Write a short, friendly email to thank a mentor for their help.",
]

for p in test_prompts:
    print("### PROMPT:", p)
    chat(p)


### PROMPT: Explain what DPO (Direct Preference Optimization) is in 2-3 sentences.


For example, you can use DPO to select a company that has the best price for your product and you don’t need to go through a lot of research. You could also choose a company that is not as good as another competitor with less competition.

4. How do you make sure your prices are correct?

You want to avoid overpaying or underpaying for your customers. If you have an app, you want to ensure that it doesn’t cost too much money on development.

Some people suggest that you pay more attention to your pricing strategy because they think that you will be able to sell your products at lower prices if you do so. However, this is not true.

If you charge too much, you will lose your customers. The only way to find out how much you should charge is to see how many people are buying your product and comparing it with other competitors.

The amount you should charge will depend on the size of


### PROMPT: Write a

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, copy

BASE_MODEL = "unsloth/smollm2-135m"  # the same one you trained from

base_tok = tokenizer  # reuse
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float32).to("cuda")

def generate(model, tok, prompt):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=150, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tok.eos_token_id)
    return tok.decode(out[0], skip_special_tokens=True)

cmp_prompt = "Give three tips to stay focused while studying."

print("----- BASE -----")
print(generate(base_model, base_tok, cmp_prompt))

print("\n----- DPO-TUNED -----")
print(generate(trainer.model, tokenizer, cmp_prompt))


----- BASE -----
Give three tips to stay focused while studying.

You need to be in a good mood

You need to be in a good mood. The best way to study is to be in a good mood. You should be relaxed and happy. If you are tired, it will be hard to concentrate. You need to be happy, happy, happy, happy.

You need to be in a good mood

You need to be in a good mood. The best way to study is to be in a good mood. You need to be happy, happy, happy, happy. You need to be happy, happy, happy, happy.

You need to be in a good mood

You need to be in a good mood. The best way to study is

----- DPO-TUNED -----
Give three tips to stay focused while studying.

1. Write down what you are trying to accomplish.
2. Write down the materials you will be studying.
3. Write down what you will do when you study.
4. Write down what you will do to help yourself study.

When you get to the point where you are done with all your studying, then you can start to review what you have learned.

Here are some tips 

In [ ]:
SAVE_DIR = "dpo_smollm2_final"
trainer.save_model(SAVE_DIR)           # saves model weights, config
tokenizer.save_pretrained(SAVE_DIR)    # saves tokenizer
print("Saved to:", SAVE_DIR)


Saved to: dpo_smollm2_final


In [ ]:
import nbformat, os

# 👇 CHANGE this to match your uploaded filename
INP  = "/content/Unsloth_ai_colab (1).ipynb"
OUTP = "/content/Unsloth_ai_colab (1).ipynb"

# ✅ Read notebook
nb = nbformat.read(INP, as_version=nbformat.NO_CONVERT)

# ✅ Remove invalid widget metadata
nb.metadata.pop("widgets", None)
for cell in nb.cells:
    cell.metadata.pop("widgets", None)

# ✅ (Optional) clear outputs for a cleaner GitHub view
for cell in nb.cells:
    if "outputs" in cell:
        cell["outputs"] = []
    if "execution_count" in cell:
        cell["execution_count"] = None

# ✅ Save cleaned version
nbformat.write(nb, OUTP)
print("Cleaned notebook saved as:", OUTP)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Unsloth_ai_colab (1).ipynb'